In [10]:
import numpy as np
import collections

In [25]:
sentences = np.load('sentences.npy')

In [26]:
sents = np.concatenate(sentences, axis=0)

In [12]:
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [32]:
vocab_size = 10000
data, count, dictionary, reverse_dictionary = build_dataset(sents, vocab_size)

In [35]:
window_size = 3
vector_dim = 300
epochs = 1000000

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [53]:
from keras.preprocessing import sequence
from keras.layers import Input, Embedding, Reshape, merge, Dense
from keras.models import Model

In [37]:
sampling_table = sequence.make_sampling_table(vocab_size)
couples, labels = sequence.skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print(couples[:10], labels[:10])

[[3359, 3507], [7547, 5786], [8005, 4422], [3552, 4020], [879, 9773], [2494, 610], [8920, 5994], [7336, 398], [6450, 6329], [2611, 9507]] [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]


In [39]:
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')

In [41]:
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

In [48]:
similarity = merge([target, context], mode='cos', dot_axes=0)

C:\Users\kaiser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
C:\Users\kaiser\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [51]:
# now perform the dot product operation to get a similarity measure
dot_product = merge([target, context], mode='dot', dot_axes=1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)

C:\Users\kaiser\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\kaiser\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [54]:
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

C:\Users\kaiser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [57]:
validation_model = Model(input=[input_target, input_context], output=similarity)

C:\Users\kaiser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("me...)`
  """Entry point for launching an IPython kernel.


In [58]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        for i in range(vocab_size):
            in_arr1[0,] = valid_word_idx
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

In [60]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 100 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    if cnt % 10000 == 0:
        sim_cb.run_sim()

Iteration 0, loss=0.6962314248085022
Nearest to 6cbbb97d-2180-4ba9-97a8-6158b6332bb4: 314ca582-bdf6-4087-95a7-e85cf604ac0e, af29874d-e74a-4517-a51c-adb0f77d6e99, 4f37dd0d-55b0-4148-b25e-4869e4f666b4, 9eb1fde0-b329-408e-9b0a-9e498d9d8a8f, 1c98375a-5b21-4b80-bf9e-81f716d409c0, 17325c07-d524-4709-9a1f-eba05b0bec1f, ed053adc-6f0c-4a81-83dd-657453cc6d22, b859db0d-2a2a-4f6b-9738-cccc808d28f7,
Nearest to 7d0632ae-5605-4ce1-b26c-b12619a2f5fb: 06805220-2af5-41df-8669-a959bb043fb1, 451119cc-90e7-498c-bae1-401822d3d3e9, 7c68a224-7681-4c14-ab02-0490ace10c78, 7e1403b4-71ac-4a8b-b810-f522e3644a79, 83743d6d-1cc7-4a76-9899-80cfd5a83780, a29d4fd2-5ed1-4af2-a1f2-01517304a00e, 18de17f3-e33d-4d86-91bc-559fbb7d5930, 3a66be2d-7c7f-4707-a495-d00b5186cdbd,
Nearest to 6830b87a-4351-4873-b2ee-80f433a522c5: ff4ef849-b212-4667-ac00-564e252bb44e, f383f6a2-f144-4d1d-a6bb-92a3decbaa1e, 9a0b6110-fa4f-49cd-be49-7238ba755b52, bc345a9b-6494-4cf5-852f-a4555bc1662d, 266ebb32-61f3-47a6-86de-0977db104dd5, 10ce3346-db65-43ca

Iteration 6400, loss=0.6877521276473999
Iteration 6500, loss=0.6881766319274902
Iteration 6600, loss=0.6813626289367676
Iteration 6700, loss=0.6883454918861389
Iteration 6800, loss=0.6898953914642334
Iteration 6900, loss=0.7101608514785767
Iteration 7000, loss=0.6663118600845337
Iteration 7100, loss=0.6769554615020752
Iteration 7200, loss=0.6813486218452454
Iteration 7300, loss=0.6774308681488037
Iteration 7400, loss=0.7078163027763367
Iteration 7500, loss=0.6866163015365601
Iteration 7600, loss=0.6917823553085327
Iteration 7700, loss=0.6901721358299255
Iteration 7800, loss=0.6907157301902771
Iteration 7900, loss=0.692674458026886
Iteration 8000, loss=0.6874052286148071
Iteration 8100, loss=0.693655252456665
Iteration 8200, loss=0.6754616498947144
Iteration 8300, loss=0.7078657150268555
Iteration 8400, loss=0.6761587858200073
Iteration 8500, loss=0.6800049543380737
Iteration 8600, loss=0.7703483700752258
Iteration 8700, loss=0.7131926417350769
Iteration 8800, loss=0.6783111095428467
It

Iteration 12800, loss=0.6789114475250244
Iteration 12900, loss=0.6772770881652832
Iteration 13000, loss=0.6755821704864502
Iteration 13100, loss=0.6729326248168945
Iteration 13200, loss=0.7090946435928345
Iteration 13300, loss=0.6828355193138123
Iteration 13400, loss=0.6779664158821106
Iteration 13500, loss=0.6852859258651733
Iteration 13600, loss=0.707269012928009
Iteration 13700, loss=0.6793797016143799
Iteration 13800, loss=0.7174186706542969
Iteration 13900, loss=0.7190083265304565
Iteration 14000, loss=0.7177055478096008
Iteration 14100, loss=0.6670289039611816
Iteration 14200, loss=0.7232794761657715
Iteration 14300, loss=0.6772053837776184
Iteration 14400, loss=0.6609770655632019
Iteration 14500, loss=0.6594018340110779
Iteration 14600, loss=0.7226499915122986
Iteration 14700, loss=0.734228253364563
Iteration 14800, loss=0.7323057651519775
Iteration 14900, loss=0.7302038669586182
Iteration 15000, loss=0.652334451675415
Iteration 15100, loss=0.6523694396018982
Iteration 15200, lo

Nearest to 64581a21-566d-4b24-99ae-c5f48b75e660: 6cb3cc94-0233-40d5-9e40-ca1699563aab, 8aa515a5-11bc-45df-a0d3-c41cb732430f, 57f7f4a8-12ef-44e2-8527-db47f09889fd, 6a8d9b8e-f785-4d40-a56d-6b6ae28a981f, e3fc34fb-e036-497b-8b7c-0d36bdb25200, 3dcf9301-f065-43d7-9886-d78758a692b7, cbae903f-fa6f-4541-b53c-7ad2554de700, dc5f6158-5e7d-4d3a-9b8e-e71b6df43fda,
Iteration 20100, loss=0.710402250289917
Iteration 20200, loss=0.6778435707092285
Iteration 20300, loss=0.6633182764053345
Iteration 20400, loss=0.721676766872406
Iteration 20500, loss=0.672417402267456
Iteration 20600, loss=0.6695536375045776
Iteration 20700, loss=0.7142037749290466
Iteration 20800, loss=0.6782793998718262
Iteration 20900, loss=0.7076436281204224
Iteration 21000, loss=0.7078973054885864
Iteration 21100, loss=0.6964647173881531
Iteration 21200, loss=0.6947661638259888
Iteration 21300, loss=0.6896135210990906
Iteration 21400, loss=0.6886758208274841
Iteration 21500, loss=0.693639874458313
Iteration 21600, loss=0.695114850997

Nearest to e5be3063-0f68-4c28-8a4e-d19a354b0872: 254505a0-49c6-4c7e-ba73-76aaceb3d1cf, cff49985-9411-4362-92d2-8d38152933b7, d9642fee-939d-4357-b697-b55f9be3a17e, aaaa662f-0d45-40c8-b727-035bd6a1ae3a, 03902941-c292-4044-87b9-b23d87d8cdbb, c0618fb4-142b-44a3-b9fc-4f32d3ba7f81, 569436a1-234a-44bc-a370-8f4d252bef21, e44bf2a8-cdd1-44c6-81db-14a0dfce206d,
Nearest to 4e6b187a-b6b6-4afd-a310-07fa9fdfd807: e5215f45-cc56-41c1-b515-4ddf388f3c06, 872937f1-e201-4b71-8085-2a1b06050fa0, a4cfa060-a5e3-47a2-baf8-d52aeb975cf7, 8896ba53-27f4-4e7b-8cd2-657e9dec48fe, bdb12265-fce1-4785-b55c-2b137bb0bfe3, 2f70665d-1daa-4bb2-805f-70fceff09559, 560b15db-79ea-49c0-b26e-84c808986e99, cb212444-c28e-4b40-b387-8b1fd7d6e014,
Nearest to 202b37a8-939a-4c60-bb1d-1f6c2e51e725: b8362114-74ab-47ac-856c-3ae48b28eb1a, 9e070507-16bc-417a-8689-f6b6050cb9e8, 5efb3f96-3d83-4173-b99c-453874bb542d, c8a2f23e-8a8c-4a97-a418-8844969f9345, 2f795d53-4034-4704-bf4b-aa54a12ba187, aa4a0633-55c8-4afb-9f02-eb5201c34b48, 2da247b2-b856-435

...

Iteration 958600, loss=0.3541122376918793
Iteration 958700, loss=1.7107123136520386
Iteration 958800, loss=0.5278342962265015
Iteration 958900, loss=1.192093321833454e-07
Iteration 959000, loss=1.192093321833454e-07
Iteration 959100, loss=0.2191884070634842
Iteration 959200, loss=1.250275731086731
Iteration 959300, loss=0.3710218071937561
Iteration 959400, loss=1.7374968528747559
Iteration 959500, loss=3.576278118089249e-07
Iteration 959600, loss=2.0559916496276855
Iteration 959700, loss=0.14191092550754547
Iteration 959800, loss=1.1562483310699463
Iteration 959900, loss=0.1669294536113739
Iteration 960000, loss=1.192093321833454e-07
Nearest to 6cbbb97d-2180-4ba9-97a8-6158b6332bb4: e4a59629-1abc-4fa4-9deb-760411368273, 0b778271-1901-472c-95fa-8996535e4c8f, fc5f3234-18b3-4218-bc73-aad533dfaeb3, 4cc4b1fc-ea44-4046-940a-da4f56425d63, 5f1477de-7c23-4218-9e21-cc50f8b9384b, a789a800-4238-4658-bd84-d39e24915239, ba4bcf45-fcdf-4ff7-a19e-be3fdffae337, 8353cabb-9649-4964-bb36-a3fd2382bae8,
Neare

Iteration 964600, loss=0.10563704371452332
Iteration 964700, loss=0.9634912610054016
Iteration 964800, loss=1.6896684169769287
Iteration 964900, loss=0.5401685833930969
Iteration 965000, loss=0.2609039843082428
Iteration 965100, loss=1.539650321006775
Iteration 965200, loss=0.16921094059944153
Iteration 965300, loss=4.8875917855184525e-06
Iteration 965400, loss=0.212666854262352
Iteration 965500, loss=1.192093321833454e-07
Iteration 965600, loss=0.19550879299640656
Iteration 965700, loss=0.35081446170806885
Iteration 965800, loss=0.023733679205179214
Iteration 965900, loss=1.5400018692016602
Iteration 966000, loss=0.9617645740509033
Iteration 966100, loss=1.0658153295516968
Iteration 966200, loss=1.2353124618530273
Iteration 966300, loss=0.00010061769717140123
Iteration 966400, loss=0.2222922295331955
Iteration 966500, loss=0.21888557076454163
Iteration 966600, loss=0.6611058712005615
Iteration 966700, loss=0.26045355200767517
Iteration 966800, loss=0.2774709165096283
Iteration 966900,

Iteration 970600, loss=0.20891061425209045
Iteration 970700, loss=0.18392063677310944
Iteration 970800, loss=0.335406631231308
Iteration 970900, loss=1.5865851640701294
Iteration 971000, loss=1.192093321833454e-07
Iteration 971100, loss=0.2739402949810028
Iteration 971200, loss=0.16414271295070648
Iteration 971300, loss=0.40029260516166687
Iteration 971400, loss=0.31451332569122314
Iteration 971500, loss=0.0739290788769722
Iteration 971600, loss=1.134749412536621
Iteration 971700, loss=0.25799572467803955
Iteration 971800, loss=0.30448582768440247
Iteration 971900, loss=0.2270108163356781
Iteration 972000, loss=0.00019009725656360388
Iteration 972100, loss=1.4305118156698882e-06
Iteration 972200, loss=0.2772712707519531
Iteration 972300, loss=0.5528528094291687
Iteration 972400, loss=1.769759178161621
Iteration 972500, loss=0.37917619943618774
Iteration 972600, loss=0.7180197238922119
Iteration 972700, loss=1.192093321833454e-07
Iteration 972800, loss=1.192093321833454e-07
Iteration 97

Nearest to 4e6b187a-b6b6-4afd-a310-07fa9fdfd807: 4e054184-6e63-426f-8f48-756d5aee466d, c5da2d84-c7d2-4bee-9416-a94394b11af1, 7f239cd8-48f9-471d-86e3-c1386a9a6d2f, 42b0da18-cb9f-45ab-afaa-72f23c9aad98, 60d43828-b3a5-46c0-940c-c0bb7f6753b5, fc18760d-8765-46c1-96e1-88c742fe33d9, f6cdcb74-06ad-4c73-a81d-204c5022e686, f5e3a2d9-29c9-4684-bf38-178c4ce76e3c,
Nearest to 202b37a8-939a-4c60-bb1d-1f6c2e51e725: b08d2a9c-17ec-4d39-9409-506d8815cfec, 153d8ca0-dc23-4548-b64f-2c49db7f30db, 1818fc80-7c01-4dda-89e1-a24851ad1838, 62ff96c2-e173-41eb-875b-061ccf7c339c, 82558949-cd98-4c58-af35-3f1a9430d52e, 20327ab0-f6c6-4401-b5c3-f7c0fc0d2bd0, 87129015-6dc5-4cfc-b71f-5e908fb01f6e, 03bd74d1-e7f9-471f-a2b0-f18fd35a4a68,
Nearest to aa6c54d3-226b-47d6-85b9-2adf3e22ebaf: d9495765-2658-4920-bb28-425e893bf19b, f75d32bb-ca82-4a5f-9cdd-554ce9f74844, 224b7b87-23c0-4ddb-b481-aa82a12fb6c0, 0d6f1e51-a121-4d54-9c4f-5d33eee08437, 1713f274-c84c-451b-9da0-f6b399f03fcc, 537d107c-569f-4d8f-a231-4e38d0d60db4, eca58753-a534-43c

Nearest to 9e2ad5bc-c6f9-40d2-a36f-3122ee2072a3: 8f5da5ab-7fc1-4dc7-a918-342af3875f5e, d6fdb107-88dc-4053-b663-6910620e1ecc, bd82738d-163c-4b1a-bfaf-7acffe30e68a, 23c3c36b-9449-4484-9040-6ef2125999aa, 47b02a82-c3bf-4647-b894-dd1c8f608e7f, 4a7fea2e-545b-4c63-bc9a-9943cc3a29d7, 63aaa8d4-93b0-4146-9554-071b715a888f, 49888c84-b4f7-491e-a93c-40c6c0393c19,
Nearest to d836830d-fb62-41b1-bc2a-fc899d1c99cc: 5a112297-02b5-4e60-939f-10274aabc541, 89a859c3-3f3e-4ff8-8cb1-bdacc8b1aecf, 1b4d1b1c-11ad-4d14-bf27-b1ce292b5c6f, cc71a15c-656f-4c14-aa15-8aafe0f3ff04, e7b82030-adb2-4534-9992-5ae822dc1d23, 80a7eaf1-4c75-4c6c-ae08-71f5d9e147ab, 8cfad2d3-ee75-4be1-b8f0-889b13c2310c, 7fdf8076-14a1-4f2d-846c-ba9e5e9710f9,
Nearest to eb612418-2d98-49af-9fdf-64362326632b: 7a3e8796-a0b3-4999-b268-4e2d47912e3c, d8c79588-fcbb-46c3-8838-85b212475bb2, 58578737-6d2e-491e-a545-6c8beb6e260c, 084a24a9-b289-4584-9fb5-1ca0f7500eb3, e6414162-7adf-4c52-b325-5361dc84a059, f5e06360-5219-455a-820e-0c53575987b5, 0313678e-237a-4a9

In [61]:
sim_cb.run_sim()

Nearest to 6cbbb97d-2180-4ba9-97a8-6158b6332bb4: e4a59629-1abc-4fa4-9deb-760411368273, 0b778271-1901-472c-95fa-8996535e4c8f, fc5f3234-18b3-4218-bc73-aad533dfaeb3, 5f1477de-7c23-4218-9e21-cc50f8b9384b, 4cc4b1fc-ea44-4046-940a-da4f56425d63, a789a800-4238-4658-bd84-d39e24915239, ba4bcf45-fcdf-4ff7-a19e-be3fdffae337, fe33d06e-ffd4-4ad9-9474-023ac977d63e,
Nearest to 7d0632ae-5605-4ce1-b26c-b12619a2f5fb: 9a671fbb-6e10-4123-a265-09fcbf7cb5e3, 2e65c66c-8c1e-453d-b08b-a5b4a9fb86bb, ed053adc-6f0c-4a81-83dd-657453cc6d22, ff4ef849-b212-4667-ac00-564e252bb44e, 17b836fe-05d2-4c0d-9312-878050d84f15, 846aa274-a2fe-43be-bd69-3c14707396cb, d143e43e-c0bb-4c5a-a6f0-ccfc448d66b5, 0c315a54-f7e1-43f9-9297-1da6cae40679,
Nearest to 6830b87a-4351-4873-b2ee-80f433a522c5: 79047824-f821-4b1a-9893-e0cea1c947dd, 5838f978-0822-4e28-874f-e1511324ec3a, ba0a796c-bd1f-4d4b-85a8-918f217a204a, c7225576-001e-423c-adc1-58f0985dcb27, 610c0012-6eb4-42a0-b759-3a2532ce0f15, 23c3c36b-9449-4484-9040-6ef2125999aa, 49888c84-b4f7-491